## 구청 홈페이지에서 정보 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# URL 목록
urls = {
    "동남구": "https://www.cheonan.go.kr/dongnam/sub05_05.do",
    "서북구": "https://www.cheonan.go.kr/seobuk/sub05_05.do"
}

rows = []

for gu, url in urls.items():
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')

    tables = soup.find_all('table')
    last_law = ''  # 〃 처리용

    for table in tables:
        headers = [th.get_text(strip=True) for th in table.find_all('th')]
        if '전화/팩스' in headers:
            cols_idx = [i for i, h in enumerate(headers) if h not in ['전화/팩스']]
        else:
            cols_idx = list(range(len(headers)))

        trs = table.find_all('tr')[1:]  # 헤더 제외

        for tr in trs:
            tds = tr.find_all(['td', 'th'])
            data = [td.get_text(strip=True) for td in tds]
            if data:
                filtered = [data[i] for i in cols_idx]
                if len(filtered) == 3:
                    name_raw = filtered[0]
                    law_raw = filtered[1]
                    addr_raw = filtered[2]

                    # 이름에서 한자/괄호 제거
                    name_clean = re.sub(r'\(.*?\)', '', name_raw).strip()
                    # 법정리/법정동에서 한자/괄호 제거
                    law_clean = re.sub(r'\(.*?\)', '', law_raw).strip()

                    # 〃 처리
                    if law_clean == '〃':
                        law_clean = last_law
                    else:
                        last_law = law_clean

                    # 우편번호 추출
                    zip_match = re.search(r'우\s*:\s*(\d+)', addr_raw)
                    zipcode = zip_match.group(1) if zip_match else ''

                    # 주소에서 (우 : ...) 제거
                    addr_clean = re.sub(r'\(우\s*:\s*\d+\)', '', addr_raw).strip()

                    row = {
                        '구': gu,
                        '구분': headers[0],
                        '이름': name_clean,
                        '법정리/법정동': law_clean,
                        '주소': addr_clean,
                        '우편번호': zipcode
                    }
                    rows.append(row)

# DataFrame 생성
df = pd.DataFrame(rows)

# 칼럼 이름 변경
df.rename(columns={'법정리/법정동': '관할 법정리'}, inplace=True)

# CSV로 저장
df.to_csv('천안시 행정구역.csv', index=False, encoding='utf-8-sig')

print(df)

      구    구분    이름                                             관할 법정리  \
0   동남구   읍·면   목천읍  교천리, 교촌리, 남화리, 덕전리, 도장리, 동리, 동평리, 삼성리, 서리, 서흥리...   
1   동남구   읍·면   풍세면             가송리, 남관리, 두남리, 미죽리, 보성리, 삼태리, 용정리, 풍서리   
2   동남구   읍·면   광덕면  광덕리, 대덕리, 대평리, 매당리, 무학리, 산원리, 신덕리, 신흥리, 원덕리,지장...   
3   동남구   읍·면    북면  납안리, 대평리, 매송리, 명덕리, 사담리,상동리, 양곡리, 연춘리, 오곡리, 용암...   
4   동남구   읍·면   성남면        대정리, 대화리, 대흥리, 봉양리, 석곡리, 신덕리, 신사리, 용원리, 화성리   
5   동남구   읍·면   수신면                       발산리, 백자리, 속창리, 신풍리, 장산리, 해정리   
6   동남구   읍·면   병천면         가전리, 관성리, 도원리, 매성리, 병천리,봉항리, 송정리, 용두리, 탑원리   
7   동남구   읍·면    동면  광덕리, 구도리, 덕성리, 동산리, 송연리, 수남리, 장송리, 죽계리, 행암리, 화...   
8   동남구  행정동명   중앙동                                 대흥동, 오룡동, 사직동, 영성동   
9   동남구  행정동명   문성동                                      문화동, 성황동, 원성동   
10  동남구  행정동명  원성1동                                            원성동,유량동   
11  동남구  행정동명  원성2동                                                원성동   
12  동남구  행정동명   봉명동                   

## 법정동 기반 주소 ➡️ 행정동 기반으로 변환 및 추출 코드

In [5]:
# 병합할 기준 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/행정구역/천안시 행정구역.csv')

In [6]:
data = data[['이름']]
data.head()

,이름
0,목천읍
1,풍세면
2,광덕면
3,북면
4,성남면


In [7]:
len(data) # 총 31개 행정동

31

In [8]:
from collections import defaultdict

# defaultdict를 사용하여 중복 key 문제 없이 다중 매핑 가능하게 함
legal_to_admin = defaultdict(list)

# 읍·면 (단일 매핑)
eup_myeon = [
    '목천읍', '풍세면', '광덕면', '북면',
    '성남면', '수신면', '병천면', '동면',
    '성환읍', '성거읍', '직산읍', '입장면'
]
for dong in eup_myeon:
    legal_to_admin[dong].append(dong)

# 동남구 행정동
legal_to_admin['대흥동'].append('중앙동')
legal_to_admin['오룡동'].append('중앙동')
legal_to_admin['사직동'].append('중앙동')
legal_to_admin['영성동'].append('중앙동')

legal_to_admin['문화동'].append('문성동')
legal_to_admin['성황동'].append('문성동')
legal_to_admin['원성동'].extend(['원성1동', '원성2동', '문성동'])  # 다중 매핑
legal_to_admin['유량동'].append('원성1동')

legal_to_admin['봉명동'].append('봉명동')

legal_to_admin['다가동'].append('일봉동')
legal_to_admin['용곡동'].extend(['일봉동', '신방동'])  # 중복 매핑
legal_to_admin['신방동'].append('신방동')

legal_to_admin['구성동'].append('청룡동')
legal_to_admin['청수동'].append('청룡동')
legal_to_admin['삼룡동'].append('청룡동')
legal_to_admin['청당동'].append('청룡동')
legal_to_admin['구룡동'].append('청룡동')

legal_to_admin['신부동'].append('신안동')
legal_to_admin['안서동'].append('신안동')

# 서북구 행정동
legal_to_admin['와촌동'].append('성정1동')
legal_to_admin['성정동'].extend(['성정1동', '성정2동'])

legal_to_admin['쌍용동'].extend(['쌍용1동', '쌍용2동', '쌍용3동'])

legal_to_admin['백석동'].append('백석동')

legal_to_admin['불당동'].extend(['불당1동', '불당2동'])

legal_to_admin['두정동'].append('부성2동')  # 구분 명확하게 하나로 설정
legal_to_admin['업성동'].append('부성1동')
legal_to_admin['신당동'].append('부성1동')
legal_to_admin['부대동'].append('부성1동')
legal_to_admin['성성동'].append('부성2동')
legal_to_admin['차암동'].append('부성2동')

# 리단위
legal_ri_map = {
    '목천읍': ['교천리', '교촌리', '남화리', '덕전리', '도장리', '동리', '동평리', '삼성리', '서리', '서흥리', '석천리', '소사리', '송전리', '신계리', '운전리', '응원리', '지산리', '천정리'],
    '풍세면': ['가송리', '남관리', '두남리', '미죽리', '보성리', '삼태리', '용정리', '풍서리'],
    '광덕면': ['광덕리', '대덕리', '대평리', '매당리', '무학리', '산원리', '신덕리', '신흥리', '원덕리', '지장리', '행정리'],
    '북면': ['납안리', '대평리', '매송리', '명덕리', '사담리', '상동리', '양곡리', '연춘리', '오곡리', '용암리', '운용리', '은지리', '전곡리'],
    '성남면': ['대정리', '대화리', '대흥리', '봉양리', '석곡리', '신덕리', '신사리', '용원리', '화성리'],
    '수신면': ['발산리', '백자리', '속창리', '신풍리', '장산리', '해정리'],
    '병천면': ['가전리', '관성리', '도원리', '매성리', '병천리', '봉항리', '송정리', '용두리', '탑원리'],
    '동면': ['광덕리', '구도리', '덕성리', '동산리', '송연리', '수남리', '장송리', '죽계리', '행암리', '화계리', '화덕리'],
    '성환읍': ['성환리', '성월리', '매주리', '율금리', '왕림리', '신방리', '송덕리', '우신리', '어룡리', '복모리', '신가리', '와룡리', '대홍리', '수향리', '안궁리', '양령리', '도하리', '학정리'],
    '성거읍': ['천흥리', '송남리', '오목리', '오색당리', '저리', '모전리', '삼곡리', '정촌리', '문덕리', '석교리', '요방리', '소우리', '신월리'],
    '직산읍': ['군동리', '군서리', '남산리', '판정리', '수헐리', '삼은리', '상덕리', '부송리', '모시리', '자은가리', '양당리', '신갈리', '마정리', '석곡리'],
    '입장면': ['하장리', '상장리', '기로리', '도림리', '양대리', '홍천리', '호당리', '시장리', '효계리', '신덕리', '신두리', '용정리', '흑암리', '산정리', '연곡리', '가산리', '유리', '독정리']
}

# 매핑 딕셔너리 생성
for admin_dong, ri_list in legal_ri_map.items():
    for legal in ri_list:
        legal_to_admin[legal].append(admin_dong)

# 완성된 딕셔너리 반환
dict(legal_to_admin)

{'목천읍': ['목천읍'],
 '풍세면': ['풍세면'],
 '광덕면': ['광덕면'],
 '북면': ['북면'],
 '성남면': ['성남면'],
 '수신면': ['수신면'],
 '병천면': ['병천면'],
 '동면': ['동면'],
 '성환읍': ['성환읍'],
 '성거읍': ['성거읍'],
 '직산읍': ['직산읍'],
 '입장면': ['입장면'],
 '대흥동': ['중앙동'],
 '오룡동': ['중앙동'],
 '사직동': ['중앙동'],
 '영성동': ['중앙동'],
 '문화동': ['문성동'],
 '성황동': ['문성동'],
 '원성동': ['원성1동', '원성2동', '문성동'],
 '유량동': ['원성1동'],
 '봉명동': ['봉명동'],
 '다가동': ['일봉동'],
 '용곡동': ['일봉동', '신방동'],
 '신방동': ['신방동'],
 '구성동': ['청룡동'],
 '청수동': ['청룡동'],
 '삼룡동': ['청룡동'],
 '청당동': ['청룡동'],
 '구룡동': ['청룡동'],
 '신부동': ['신안동'],
 '안서동': ['신안동'],
 '와촌동': ['성정1동'],
 '성정동': ['성정1동', '성정2동'],
 '쌍용동': ['쌍용1동', '쌍용2동', '쌍용3동'],
 '백석동': ['백석동'],
 '불당동': ['불당1동', '불당2동'],
 '두정동': ['부성2동'],
 '업성동': ['부성1동'],
 '신당동': ['부성1동'],
 '부대동': ['부성1동'],
 '성성동': ['부성2동'],
 '차암동': ['부성2동'],
 '교천리': ['목천읍'],
 '교촌리': ['목천읍'],
 '남화리': ['목천읍'],
 '덕전리': ['목천읍'],
 '도장리': ['목천읍'],
 '동리': ['목천읍'],
 '동평리': ['목천읍'],
 '삼성리': ['목천읍'],
 '서리': ['목천읍'],
 '서흥리': ['목천읍'],
 '석천리': ['목천읍'],
 '소사리': ['목천읍'],
 '송전리': ['목천읍'],
 '신계리

In [9]:
# 법정동 추출 함수
def extract_legal_dong(address):
    for legal_dong in legal_to_admin.keys():
        if legal_dong in address:
            return legal_dong
    return None

# 행정동 매핑 함수: 다중 매핑 리스트 반환, 없으면 빈 리스트 반환
def map_to_admin_dong_all(address):
    legal_dong = extract_legal_dong(address)
    if legal_dong:
        return legal_to_admin[legal_dong]
    return []